#  Deaths heatmap
---

Inspired by: https://twitter.com/VictimOfMaths/status/1258783876870176770?s=09

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Import-data" data-toc-modified-id="Import-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Draft:-Cities-heatmap" data-toc-modified-id="Draft:-Cities-heatmap-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Draft: Cities heatmap</a></span></li></ul></div>

## Import packages

In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import yaml
config = yaml.load(open('../src/configs/config.yaml', 'r'))

/Users/fernandascovino/Projects/qualsetorreabrir/venvanalysis/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [2]:
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

## Import data

In [3]:
# df = pd.read_csv('data/raw/br_cases_city_daily_notification_rate.csv', index_col=0)

In [8]:
# 1. Abra o simulacovid-datasource
# 2. Rode make loader-build-run para gerar a API

df = pd.read_csv('http://localhost:7000/v1/raw/csv')
df

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,city,last_updated,epidemiological_week,estimated_population_2019,is_last,is_repeated,confirmed_cases,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,deaths,order_for_place,place_type,state,daily_cases,new_deaths,infectious_period_cases,state_notification_rate,notification_rate,active_cases,data_last_refreshed
0,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,0.0,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,0.0,Acrelândia,2020-03-29 00:00:00,14.0,15256.0,False,False,2.0,13.10960,2020-03-29,0.0,0.0,1.0,city,AC,2.0,0.0,2.0,0.591955,0.382857,5.0,2020-05-11 16:17:51
1,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,0.0,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,0.0,Acrelândia,2020-03-30 00:00:00,14.0,15256.0,False,False,6.0,39.32879,2020-03-30,0.0,0.0,2.0,city,AC,4.0,0.0,6.0,0.591955,0.382857,16.0,2020-05-11 16:17:51
2,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,0.0,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,0.0,Acrelândia,2020-03-31 00:00:00,14.0,15256.0,False,False,7.0,45.88359,2020-03-31,0.0,0.0,3.0,city,AC,1.0,0.0,7.0,0.591955,0.382857,18.0,2020-05-11 16:17:51
3,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,0.0,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,0.0,Acrelândia,2020-04-01 00:00:00,14.0,15256.0,False,False,8.0,52.43838,2020-04-01,0.0,0.0,4.0,city,AC,1.0,0.0,8.0,0.591955,0.382857,21.0,2020-05-11 16:17:51
4,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,0.0,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,0.0,Acrelândia,2020-04-02 00:00:00,14.0,15256.0,False,False,8.0,52.43838,2020-04-02,0.0,0.0,5.0,city,AC,0.0,0.0,8.0,0.591955,0.382857,21.0,2020-05-11 16:17:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68452,BR,Brasil,TO,Tocantins,1722081,Wanderlândia,11683,Região de Saúde Médio Norte,2020-02-01,SUS,0.0,2020-02-01,SUS,0.0,2020-02-01,SUS,0.0,Wanderlândia,2020-05-08 00:00:00,19.0,11683.0,False,False,8.0,68.47556,2020-05-08,0.0,0.0,11.0,city,TO,1.0,0.0,8.0,1.000000,1.000000,8.0,2020-05-11 16:17:51
68453,BR,Brasil,TO,Tocantins,1722081,Wanderlândia,11683,Região de Saúde Médio Norte,2020-02-01,SUS,0.0,2020-02-01,SUS,0.0,2020-02-01,SUS,0.0,Wanderlândia,2020-05-09 00:00:00,19.0,11683.0,True,False,8.0,68.47556,2020-05-09,0.0,0.0,12.0,city,TO,0.0,0.0,8.0,1.000000,1.000000,8.0,2020-05-11 16:17:51
68454,BR,Brasil,TO,Tocantins,1722081,Wanderlândia,11683,Região de Saúde Médio Norte,2020-02-01,SUS,0.0,2020-02-01,SUS,0.0,2020-02-01,SUS,0.0,Wanderlândia,2020-05-10 00:00:00,20.0,11683.0,False,True,8.0,68.47556,2020-05-09,0.0,0.0,13.0,city,TO,0.0,0.0,8.0,1.000000,1.000000,8.0,2020-05-11 16:17:51
68455,BR,Brasil,TO,Tocantins,1722107,Xambioá,11540,Região de Saúde do Médio Araguaia,2020-02-01,SUS,0.0,2020-02-01,SUS,5.6,2020-02-01,SUS,0.0,Xambioá,2020-05-09 00:00:00,19.0,11540.0,True,False,2.0,17.33102,2020-05-09,0.0,0.0,1.0,city,TO,2.0,0.0,2.0,1.000000,1.000000,2.0,2020-05-11 16:17:51


In [36]:
import loader
import yaml
import plotly.graph_objs as go
import plotly.offline as plt
from plotly.io import to_html
import numpy as np
import pandas as pd

import utils

def _df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}
    
def plot_deaths_heatmap(t, place_type, title=None):
    
    # group df
    df_heatmap = t.reset_index()\
                    .pivot(index=place_type, 
                            columns='last_updated', 
                            values='rolling_deaths_new')\
                    .fillna(0)\
                    .apply(lambda x: x/x.max(), axis=1)\
                    .dropna(how='all')

    # remove days with all states zero
    df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
    
    # sort by date of max value ==> first on 0.5
    d = df_heatmap.mask(df_heatmap < 0.5)
    df_heatmap['order'] = d.apply(pd.Series.first_valid_index, 1)
    # df_heatmap['order'] = df_heatmap.idxmax(axis=1)

    df_heatmap = df_heatmap.sort_values('order', ascending=False)
    del df_heatmap['order']
    
    df_heatmap = _df_to_plotly(df_heatmap)

    # style hovertext
    hovertext = list()
    for yi, yy in enumerate(df_heatmap['y']):
        hovertext.append(list())
        for xi, xx in enumerate(df_heatmap['x']):
            hovertext[-1].append('Data: {}<br />UF: {}<br />Média: {:.2f}'.format(xx, yy, df_heatmap['z'][yi][xi]))
    
    # generate fig
    fig = go.Figure(data=go.Heatmap(
                                df_heatmap,
                                colorscale='temps',
                                showscale=False,
                                text=hovertext,
                                hoverinfo='text',
                            ),
                    layout={
                        # 'autosize': True,
                        'width': 1000,
                        'height': 700, 
                        'margin': {'l': 500, 't': 0}
                    }
                )

    legend = """
        O gráfico ao lado mostra a média do número de mortes<br>
        diárias dos últimos cinco dias em cada UF, desde a data da<br>
        primeira morte reportada. Para comparação, os números foram<br>
        normalizados pelo maior valor encontrado em cada UF:<br>
        <b>quanto mais vermelho, mais próximo está o valor do<br>
        maior número de mortes por dia observado na UF até hoje</b>.<br>
        <br>
        As UFs estão ordenadas pela data em que atingiram <br>
        metade do número máximo de mortes diário. Ou seja, a data<br>
        em que foi observado metade do maior número de mortes em <i>{}</i><br>
        é anterior à data das demais UFs.
        <br><br>
        Última atualização: {}
    """.format(df_heatmap['y'][-1], df_heatmap['x'][-1])

    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-1.3,
                            y=1,
                            showarrow=False,
                             text=legend,
                            xref="paper",
                            yref="paper",
                            align='left'
                           ))

    return fig

def _get_rolling_amount(grp, days):
    return grp.rolling(days, min_periods=1, on='last_updated')['deaths'].mean()

def prepare_heatmap(df, mavg_days=5):

    df_deaths = df[~df['deaths'].isnull()][['state', 'last_updated', 'deaths']]\
                                                .groupby(['state', 'last_updated'])['deaths']\
                                                .sum()\
                                                .reset_index()

    df_deaths['rolling_deaths_new'] = df_deaths.groupby('state', 
                                                    as_index=False, 
                                                    group_keys=False)\
                                           .apply(lambda x : _get_rolling_amount(x, mavg_days))

    return plot_deaths_heatmap(df_deaths, 'state')

In [37]:
prepare_heatmap(df)

## Draft: Cities heatmap

In [ ]:
df_heatmap = state_df_deaths.reset_index()\
                      .pivot(index='state', 
                              columns='date', 
                              values='rolling_deaths_new')\
                       .fillna(0)\
                       .apply(lambda x: x/x.max(), axis=1)

# remove days with all states zero
df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]

def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

fig = go.Figure(data=go.Heatmap(df_to_plotly(df_heatmap), colorscale='temps'), 
                layout={'width': 600,
                        'height': 600,
                        'title': 'Distribuição de novas mortes nas UFs (mavg = 5 days)'})
fig.show()

In [20]:
state = 'RS'

In [21]:
df_deaths = df[(~df['deaths'].isnull()) & (df['state'] == state)][['city', 
                                                                 'last_updated', 
                                                                 'deaths']]

df_deaths['rolling_deaths_new'] = df_deaths.groupby('city', 
                                                as_index=False, 
                                                group_keys=False)\
                                          .apply(get_rolling_amount)

In [23]:
plot_deaths_heatmap(df_deaths, 'city', 'Distribuição de novas mortes em {}'.format(state))

30
